In [10]:
!pip install pandas
!pip install numpy
!pip install re
!pip install os
!pip install matplotlib

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [11]:
'''PART 1: COMBINE OUTPUT SIMULATIONS IN A BIGGER DATASET FOR ANALYSIS.'''

import pandas as pd
import re  
import os

# Path to the folder containing your files
folder_path = 'output_model/'

# List to store all file paths
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize two lists to store dataframes by density
dfs_high = []
dfs_low = []

# Function to extract repetition, density, and architecture from the filename
def extract_simulation_info(file_path):
    match = re.search(r'repetition_(\d+)_(High|Low)_(architecture_[A-I]|control)', file_path)
    if match:
        return match.group(1), match.group(2), match.group(3)  # repetition, density, architecture
    else:
        return 'Unknown', 'Unknown', 'Unknown'

# Loop through each file
for file in file_paths:
    # Load the file
    df = pd.read_csv(file)
    
    # Adjust the 'rank' column by subtracting 1 from each value to start ranks from 1
    df['rank'] = df['rank'] - 1

    # Remove rows where Organ_ID starts with "organs.Internode"
    df = df[~df['Organ_ID'].str.startswith('organs.Internode')]

    # Extract repetition, density, and architecture from the filename
    repetition, density, architecture = extract_simulation_info(file)

    # Add the extracted information as new columns
    df['density'] = density
    df['architecture'] = architecture
    df['repetition'] = repetition  

    # Drop unnecessary columns
    columns_to_drop = [
        'organ_type', 'age_in_degree_days_dd of plant', 'age_in_days_d of plant',
        'order', 'dry_biomass_mg[mg]', 'dry_biomass_growth_mg for dynamic', 'SLA'
    ]
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Create new column for absorbedPAR [umol m^-2 s^-1]
    df['absorbedPAR_umol_m2_s1'] = df['absorbedPAR [umol s^-1]'] / df['area_m2[m^2]']

    # Handle rank 1 separately by first summing the values for two leaves and then averaging
    rank1_df = df[df['rank'] == 1]
    numeric_columns = ['absorbedPAR_umol_m2_s1', 'area_m2[m^2]', 'absorbedPAR [umol s^-1]']
    rank1_summed = rank1_df.groupby(['density', 'architecture', 'repetition', 'plantNb']).sum().reset_index()
    rank1_averaged = rank1_summed.groupby(['density', 'architecture', 'repetition'])[numeric_columns].mean().reset_index()
    rank1_averaged['rank'] = 1 
    
    # Handle other ranks normally
    other_ranks_df = df[df['rank'] != 1]
    other_ranks_mean = other_ranks_df.groupby(['density', 'architecture', 'repetition', 'rank'])[numeric_columns].mean().reset_index()

    # Combine rank 2 and other ranks
    combined_df = pd.concat([rank1_averaged, other_ranks_mean], ignore_index=True)
    
    # Append the processed DataFrame to the appropriate list based on density
    if density == 'High':
        dfs_high.append(combined_df)
    elif density == 'Low':
        dfs_low.append(combined_df)

# Concatenate all High density dataframes into one
if dfs_high:  # Check if there are any dataframes to concatenate
    combined_high = pd.concat(dfs_high, ignore_index=True)
    combined_high = combined_high[['density', 'architecture', 'repetition', 'rank', 'area_m2[m^2]', 'absorbedPAR_umol_m2_s1']]
    combined_high.to_csv('combined_files/combined_high_ranks.csv', index=False)
    print("High density data processed and saved.")
else:
    print("No High density data found. Skipping high density processing.")

# Concatenate all Low density dataframes into one
if dfs_low:  # Check if there are any dataframes to concatenate
    combined_low = pd.concat(dfs_low, ignore_index=True)
    combined_low = combined_low[['density', 'architecture', 'repetition', 'rank', 'area_m2[m^2]', 'absorbedPAR_umol_m2_s1']]
    combined_low.to_csv('combined_files/combined_low_ranks.csv', index=False)
    print("Low density data processed and saved.")
else:
    print("No Low density data found. Skipping low density processing.")

High density data processed and saved.
No Low density data found. Skipping low density processing.


In [12]:
'''PART 2: CREATE DATASET FOR TOTAL ABSORBED PAR FOR EACH PLANT IN HIGH AND LOW DENSITY.'''

import pandas as pd

# Define a function to process and aggregate the data
def aggregate_absorbed_PAR(file_path, output_path):
    # Load the data
    data = pd.read_csv(file_path)
    
    # Group the data by 'density', 'architecture', 'repetition' and sum the specified columns
    grouped_data = data.groupby(['density', 'architecture', 'repetition']).agg({
        'absorbedPAR_umol_m2_s1': 'sum',
        'area_m2[m^2]': 'sum'
    }).reset_index()
    
    # Save the aggregated data to a new CSV file
    grouped_data.to_csv(output_path, index=False)
    print(f"Aggregation complete. Data saved to: {output_path}")

# File paths for high and low density data
file_paths = {
    'high': ('combined_files/combined_high_ranks.csv', 'combined_files/combined_total_absorbedPAR_high.csv'),
    'low': ('combined_files/combined_low_ranks.csv', 'combined_files/combined_total_absorbedPAR_low.csv')
}

# Loop through both high and low files and apply the aggregation
for density, (input_file, output_file) in file_paths.items():
    aggregate_absorbed_PAR(input_file, output_file)


Aggregation complete. Data saved to: combined_files/combined_total_absorbedPAR_high.csv
Aggregation complete. Data saved to: combined_files/combined_total_absorbedPAR_low.csv


In [13]:
'''PART 3: COMBINE RESULTS SENSORS FROM OUTPUT SIMULATION IN A BIGGER DATASET.'''

import pandas as pd
import re
import os

# Specify folder path and create a list of all csv files contained in the folder path.
folder_path = "output_sensors/"
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize two lists to store dataframes by density
dfs_high = []
dfs_low = []

# Function to extract repetition, density, and architecture from the filename
def extract_simulation_info(file_path):
    match = re.search(r'sensors_export_below_canopy\.csv_repetition_(\d+)_(High|Low)_(architecture_[A-I]|control)', file_path)
    if match:
        return match.group(1), match.group(2), match.group(3)  # repetition, density, architecture
    else:
        return 'Unknown', 'Unknown', 'Unknown'
    
# Loop through each file.
for file in file_paths:
    # Load the file into a pandas dataframe.
    df = pd.read_csv(file)
    
    # Calculate mean of all rows for the specified numeric columns
    numeric_columns = [' Tile z cohordinate [m]', 'absorbedPARTile [umol^s-1]']
    mean_values = df[numeric_columns].mean()  # Mean values for each numeric column
    
    # Convert mean_values to a DataFrame to keep a single row
    mean_df = pd.DataFrame([mean_values])

    # Extract repetition, density, and architecture from the filename
    repetition, density, architecture = extract_simulation_info(file)

    # Add the extracted information as new columns
    mean_df['density'] = density
    mean_df['architecture'] = architecture
    mean_df['repetition'] = repetition  
    
    # Calculate the new column for absorbedPAR micromols m-2 s-1 (using mean values)
    mean_df['absorbedPAR_umol_m2_s1'] = mean_df['absorbedPARTile [umol^s-1]'] / mean_df[' Tile z cohordinate [m]']
    
    # Drop unnecessary columns from the mean_df
    mean_df.drop(columns=[' Tile z cohordinate [m]', 'absorbedPARTile [umol^s-1]'], inplace=True)
    
    # Append the processed DataFrame to the appropriate list based on density
    if density == 'High':
        dfs_high.append(mean_df)
    elif density == 'Low':
        dfs_low.append(mean_df)

# Concatenate all high and low density dataframes if they are not empty
if dfs_high:  # Check if there are any dataframes in the high density list
    combined_high = pd.concat(dfs_high, ignore_index=True)
    combined_high = combined_high[['density', 'architecture', 'repetition', 'absorbedPAR_umol_m2_s1']]
    # Save the combined high density data to a new CSV
    combined_high.to_csv('combined_files/combined_high_sensors.csv', index=False)
    print("High density data processed and saved.")
else:
    print("No High density data to process. Skipping high density processing.")

if dfs_low:  # Check if there are any dataframes in the low density list
    combined_low = pd.concat(dfs_low, ignore_index=True)
    combined_low = combined_low[['density', 'architecture', 'repetition', 'absorbedPAR_umol_m2_s1']]
    # Save the combined low density data to a new CSV
    combined_low.to_csv('combined_files/combined_low_sensors.csv', index=False)
    print("Low density data processed and saved.")
else:
    print("No Low density data to process. Skipping low density processing.")

print("Data processing complete.")

High density data processed and saved.
No Low density data to process. Skipping low density processing.
Data processing complete.


In [19]:
'''PART 4: CHECK AND ELIMINATE OUTLIERS, CREATE BOXPLOT OF DATA FOR RANKS (ORIGINAL AND CLEANED).'''
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# Function to process data, detect outliers, generate boxplots, and save results
def process_ranks_data(input_file, output_cleaned_file, output_outliers_file, output_boxplot_folder, density_label):
    # Create folder for boxplots if it doesn't exist
    if not os.path.exists(output_boxplot_folder):
        os.makedirs(output_boxplot_folder)

    # Load the dataset
    data = pd.read_csv(input_file)

    # Initialize empty DataFrames to store cleaned data and outliers
    cleaned_data = pd.DataFrame()
    outliers = pd.DataFrame()

    # Process data by rank
    for rank in data['rank'].unique():
        # Filter data for the current rank
        rank_data = data[data['rank'] == rank]
        
        # Detect and remove outliers for each architecture within this rank
        for architecture in rank_data['architecture'].unique():
            # Filter data for the current architecture within the current rank
            arch_data = rank_data[rank_data['architecture'] == architecture]

            # Calculate Q1, Q3, and IQR
            multiplier = 1.5
            Q1 = arch_data['absorbedPAR_umol_m2_s1'].quantile(0.25)
            Q3 = arch_data['absorbedPAR_umol_m2_s1'].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR

            # Identify and remove outliers
            is_outlier = (arch_data['absorbedPAR_umol_m2_s1'] < lower_bound) | (arch_data['absorbedPAR_umol_m2_s1'] > upper_bound)
            arch_outliers = arch_data[is_outlier]
            arch_data_cleaned = arch_data[~is_outlier]

            # Append the cleaned data and outliers
            cleaned_data = pd.concat([cleaned_data, arch_data_cleaned], ignore_index=True)
            outliers = pd.concat([outliers, arch_outliers], ignore_index=True)

        # Generate and save boxplot with outliers for the current rank
        plt.figure(figsize=(10, 6))
        rank_data.boxplot(column='absorbedPAR_umol_m2_s1', by='architecture', grid=False)
        plt.title(f'Boxplot of Absorbed PAR by Architecture for Rank {rank} ({density_label})')
        plt.suptitle('')
        plt.xticks(fontsize=5)
        plt.ylabel('Absorbed PAR (umol m-2 s-1)')
        plt.savefig(os.path.join(output_boxplot_folder, f'{density_label}_rank_{rank}_with_outliers.png'))
        plt.close()

    # Save the cleaned data and outliers to CSV files
    cleaned_data.to_csv(output_cleaned_file, index=False)
    outliers.to_csv(output_outliers_file, index=False)

    # Print summary
    print(f"Outliers removed and saved to '{output_outliers_file}'")
    print(f"Cleaned data saved to '{output_cleaned_file}'")
    print(f"Boxplots with outliers saved in '{output_boxplot_folder}'")

    # Generate and save boxplots for cleaned data
    for rank in cleaned_data['rank'].unique():
        rank_data_cleaned = cleaned_data[cleaned_data['rank'] == rank]
        plt.figure(figsize=(10, 6))
        rank_data_cleaned.boxplot(column='absorbedPAR_umol_m2_s1', by='architecture', grid=False)
        plt.title(f'Cleaned Boxplot of Absorbed PAR by Architecture for Rank {rank} ({density_label})')
        plt.suptitle('')
        plt.xticks(fontsize=5)
        plt.ylabel('Absorbed PAR (umol m-2 s-1)')
        plt.savefig(os.path.join(output_boxplot_folder, f'{density_label}_rank_{rank}_cleaned.png'))
        plt.close()

    print(f"Cleaned boxplots saved in '{output_boxplot_folder}'")


# Define file paths and labels for high and low density datasets
datasets = [
    {
        'input_file': 'combined_files/combined_high_ranks.csv',
        'output_cleaned_file': 'combined_files/combined_high_ranks_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_high_ranks_outliers.csv',
        'output_boxplot_folder': 'boxplots/high_ranks',
        'label': 'High Density'
    },
    {
        'input_file': 'combined_files/combined_low_ranks.csv',
        'output_cleaned_file': 'combined_files/combined_low_ranks_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_low_ranks_outliers.csv',
        'output_boxplot_folder': 'boxplots/low_ranks',
        'label': 'Low Density'
    }
]

# Process each dataset using the function
for dataset in datasets:
    process_ranks_data(
        dataset['input_file'],
        dataset['output_cleaned_file'],
        dataset['output_outliers_file'],
        dataset['output_boxplot_folder'],
        dataset['label']
    )

print("Processing complete for all datasets.")

Outliers removed and saved to 'combined_files/combined_high_ranks_outliers.csv'
Cleaned data saved to 'combined_files/combined_high_ranks_cleaned.csv'
Boxplots with outliers saved in 'boxplots/high_ranks'


c:\Users\jacop\OneDrive\Desktop\Thesis\Data analysis\Statistics\venv\Lib\site-packages\pandas\plotting\_matplotlib\tools.py:233: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(**fig_kw)


Cleaned boxplots saved in 'boxplots/high_ranks'


C:\Users\jacop\AppData\Local\Temp\ipykernel_11304\1035860870.py:48: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 6))


Outliers removed and saved to 'combined_files/combined_low_ranks_outliers.csv'
Cleaned data saved to 'combined_files/combined_low_ranks_cleaned.csv'
Boxplots with outliers saved in 'boxplots/low_ranks'
Cleaned boxplots saved in 'boxplots/low_ranks'
Processing complete for all datasets.


...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [20]:
'''PART 5: OUTLIERS CHECK AND BOXPLOT GENERATION FOR ORIGINAL AND CLEANED DATA OF SENSORS AND TOTAL ABSORBED PAR PER PLANT.'''
import pandas as pd 
import os
import matplotlib.pyplot as plt

# Function to detect outliers, generate boxplots (original and cleaned data), and save results
def handle_outliers_and_boxplots(input_file, output_cleaned_file, output_outliers_file, output_boxplot_folder, dataset_label):
    # Create folder for boxplots if it doesn't exist
    if not os.path.exists(output_boxplot_folder):
        os.makedirs(output_boxplot_folder)

    # Load the dataset
    data = pd.read_csv(input_file)

    # Initialize empty DataFrames to store cleaned data and outliers
    cleaned_data = pd.DataFrame()
    outliers = pd.DataFrame()

    # Detect and remove outliers for each architecture
    for architecture in data['architecture'].unique():
        # Filter data for the current architecture
        arch_data = data[data['architecture'] == architecture]

        # Calculate Q1, Q3, and IQR
        multiplier = 1.5
        Q1 = arch_data['absorbedPAR_umol_m2_s1'].quantile(0.25)
        Q3 = arch_data['absorbedPAR_umol_m2_s1'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR

        # Identify outliers and filter them
        is_outlier = (arch_data['absorbedPAR_umol_m2_s1'] < lower_bound) | (arch_data['absorbedPAR_umol_m2_s1'] > upper_bound)
        arch_outliers = arch_data[is_outlier]
        arch_data_cleaned = arch_data[~is_outlier]

        # Append the cleaned data and outliers
        cleaned_data = pd.concat([cleaned_data, arch_data_cleaned], ignore_index=True)
        outliers = pd.concat([outliers, arch_outliers], ignore_index=True)

    # Generate and save original boxplot (before outlier removal)
    plt.figure(figsize=(10, 6))
    data.boxplot(column='absorbedPAR_umol_m2_s1', by='architecture', grid=False)
    plt.title(f'Original Boxplot of Absorbed PAR ({dataset_label})')
    plt.suptitle('')
    plt.xticks(fontsize=5)
    plt.ylabel('Absorbed PAR (umol m-2 s-1)')
    plt.savefig(os.path.join(output_boxplot_folder, f'{dataset_label}_original.png'))
    plt.close()

    # Generate and save cleaned boxplot (after outlier removal)
    plt.figure(figsize=(10, 6))
    cleaned_data.boxplot(column='absorbedPAR_umol_m2_s1', by='architecture', grid=False)
    plt.title(f'Cleaned Boxplot of Absorbed PAR ({dataset_label})')
    plt.suptitle('')
    plt.xticks(fontsize=5)
    plt.ylabel('Absorbed PAR (umol m-2 s-1)')
    plt.savefig(os.path.join(output_boxplot_folder, f'{dataset_label}_cleaned.png'))
    plt.close()

    # Save the cleaned data and outliers
    cleaned_data.to_csv(output_cleaned_file, index=False)
    outliers.to_csv(output_outliers_file, index=False)

    # Print summary
    print(f"Outliers detected and saved to '{output_outliers_file}'")
    print(f"Cleaned data saved to '{output_cleaned_file}'")
    print(f"Boxplots saved in '{output_boxplot_folder}'")


# Define file paths and labels for high and low density datasets
datasets = [
    {
        'input_file': 'combined_files/combined_high_sensors.csv',
        'output_cleaned_file': 'combined_files/combined_high_sensors_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_high_sensors_outliers.csv',
        'output_boxplot_folder': 'boxplots/high_sensors',
        'label': 'High Density'
    },
    {
        'input_file': 'combined_files/combined_low_sensors.csv',
        'output_cleaned_file': 'combined_files/combined_low_sensors_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_low_sensors_outliers.csv',
        'output_boxplot_folder': 'boxplots/low_sensors',
        'label': 'Low Density'
    },
        {
        'input_file': 'combined_files/combined_total_absorbedPAR_high.csv',
        'output_cleaned_file': 'combined_files/combined_total_absorbedPAR_high_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_total_absorbedPAR_high_outliers.csv',
        'output_boxplot_folder': 'boxplots/total_absorbedPAR_high',
        'label': 'High Density'
    },
    {
        'input_file': 'combined_files/combined_total_absorbedPAR_low.csv',
        'output_cleaned_file': 'combined_files/combined_total_absorbedPAR_low_cleaned.csv',
        'output_outliers_file': 'combined_files/combined_total_absorbedPAR_low_outliers.csv',
        'output_boxplot_folder': 'boxplots/total_absorbedPAR_low',
        'label': 'Low Density'
    }
]

# Process each dataset using the function
for dataset in datasets:
    handle_outliers_and_boxplots(
        dataset['input_file'],
        dataset['output_cleaned_file'],
        dataset['output_outliers_file'],
        dataset['output_boxplot_folder'],
        dataset['label']
    )

print("Processing complete for all datasets.")

Outliers detected and saved to 'combined_files/combined_high_sensors_outliers.csv'
Cleaned data saved to 'combined_files/combined_high_sensors_cleaned.csv'
Boxplots saved in 'boxplots/high_sensors'
Outliers detected and saved to 'combined_files/combined_low_sensors_outliers.csv'
Cleaned data saved to 'combined_files/combined_low_sensors_cleaned.csv'
Boxplots saved in 'boxplots/low_sensors'
Outliers detected and saved to 'combined_files/combined_total_absorbedPAR_high_outliers.csv'
Cleaned data saved to 'combined_files/combined_total_absorbedPAR_high_cleaned.csv'
Boxplots saved in 'boxplots/total_absorbedPAR_high'
Outliers detected and saved to 'combined_files/combined_total_absorbedPAR_low_outliers.csv'
Cleaned data saved to 'combined_files/combined_total_absorbedPAR_low_cleaned.csv'
Boxplots saved in 'boxplots/total_absorbedPAR_low'
Processing complete for all datasets.


<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

In [16]:
'''PART 6: LOG TRANFORMATION OF DATA.'''

import os
import pandas as pd
import numpy as np

# Define the folder paths
input_folder = 'combined_files/'
output_folder = 'log_transformed/'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through the files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):
        # Load the CSV file
        file_path = os.path.join(input_folder, file_name)
        data = pd.read_csv(file_path)

        # Log transformation (base 10) of the 'absorbedPAR_umol_m2_s1' column
        if 'absorbedPAR_umol_m2_s1' in data.columns:
            # Apply log transformation
            data['log_absorbedPAR_umol_m2_s1'] = np.log10(data['absorbedPAR_umol_m2_s1'])

            # Save the transformed data to a new file in the output folder
            output_file_path = os.path.join(output_folder, file_name)
            data.to_csv(output_file_path, index=False)

            print(f"Log transformation applied and saved for {file_name}")

print("All files processed and saved in 'log_transformed' folder.")


Log transformation applied and saved for combined_high_ranks.csv
Log transformation applied and saved for combined_high_ranks_cleaned.csv
Log transformation applied and saved for combined_high_ranks_outliers.csv
Log transformation applied and saved for combined_high_sensors.csv
Log transformation applied and saved for combined_high_sensors_cleaned.csv
Log transformation applied and saved for combined_high_sensors_outliers.csv
Log transformation applied and saved for combined_low_ranks.csv
Log transformation applied and saved for combined_low_ranks_cleaned.csv
Log transformation applied and saved for combined_low_ranks_outliers.csv
Log transformation applied and saved for combined_low_sensors.csv
Log transformation applied and saved for combined_low_sensors_cleaned.csv
Log transformation applied and saved for combined_low_sensors_outliers.csv
Log transformation applied and saved for combined_total_absorbedPAR_high.csv
Log transformation applied and saved for combined_total_absorbedPAR_h